Im ersten Block erfolgt das Importieren der benötigten Bibliotheken:

- Standardbibliothek "csv" um eine CSV einzulesen
- "re"-Bibliothek um unsauberen Text zu bereinigen
- mit den "NLTK"-Bibliotheken zum Herausfiltern von Stoppwörtern, Vektorisierung mit CountVectorizer, TF-IDF-Vektor und Extrahieren von Themen
- "TextBlob"-Bibliothek für die Sentiment Analyse


In [ ]:
import csv
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from textblob import TextBlob

Im folgenden Block wird der Datensatz (CSV-Datei) geöffnet und eingelesen. Anschließend wird eine Liste "no_stopword_list" erstellt, um die später bereinigten Wörter hinzuzufügen. Ebenso werden drei Zähler erstellt, jeweils um die Sentiment-Reviews zu zählen.

In der For-Schleife wird für jede Zeile die Spalte "Customer Complaint" ausgewählt und anschließend:

- die Sonderzeichen und Satzzeichen mit der "re"-Bibliothek bereinigt
- alle Buchstaben mit der Funkion "lower()" in Kleinbuchstaben umgewandelt
- der Text tokenisiert, um einzelne Wörter zu erhalten.

In dieser For-Schleife wird auch die Sentiment-Analyse durchgeführt, um den Stimmungs-Wert jedes einzelnen Beschwerde-Eintrags zu erhalten:

- Initialisieren des TextBlob-Objekts
- Umwandeln des rohen Textes in ein String-Objekt
- Durchführung der Sentiment-Analyse
- Ausgabe der einzelnen Sentiment-Werte

Mit einer If-Verzweigung werden die Reviews eingeordnet und die entsprechenden Zähler hochgezählt.

Abschließend wird noch eine Liste erstellt mit englischen Stoppwörtern. Mithilfe einer weiteren For-Schleife werden die einzelnen Wörter untersucht, ob sie Stoppwörter sind. Wenn nicht, dann werden sie an die "no_stopword_list" angehängt.

Zur Rückmeldung dieses Abschnittes werden die Ergebnisse der Sentiment-Analyse, also die Zähler der drei Stimmungen, ausgegeben.


In [ ]:
with open('Comcast.csv', newline='') as csvfile:
    raw_text = csv.DictReader(csvfile)

    no_stopword_list = []
    
    negative_count = 0
    neutral_count = 0
    positive_count = 0

    for row in raw_text:
        text_list = row['Customer Complaint']
        text_list = re.sub('[^a-zA-Z0-9 ]', '', str(text_list))
        text_list = text_list.lower()
        text_list = text_list.split()

        blob = TextBlob(str(text_list))
        sentiment = blob.sentiment.polarity
        print("Sentiment:", sentiment)

        if sentiment < 0:
            negative_count += 1
        elif sentiment == 0:
            neutral_count += 1
        else:
            positive_count += 1

        stop_words_list = set(stopwords.words('english'))
        for word in text_list:
            if word not in stop_words_list:
                no_stopword_list.append(word)

    print("\nAnzahl negativer Reviews:", negative_count)
    print("Anzahl neutraler Reviews:", neutral_count)
    print("Anzahl positiver Reviews:", positive_count)
    

Der folgende Abschnitt wandelt die bereinigten Wörter in numerische Vektoren um. Hier werden zwei Varianten umgesetzt:

- CountVectorizer
- Term Frequency - Inverse Document Frequency (TF-IDF)

Beide werden jeweils zuerst initialisiert, um dann auf die Wortliste angewandt zu werden,
die keine Stoppwörter mehr enthält.

Als Rückmeldung werden die einzelnen Vektoren ausgegeben, so können die Ergebnisse dieses Abschnitts verglichen werden.


In [ ]:
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(no_stopword_list)
print("CountVectorizer-Ergebnisse:\n", vectors)

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(no_stopword_list)
print("\nTF-IDF-Ergebnisse:\n", tfidf)


Im letzten Block wird die Themenmodellierung ausgeführt und die häufigsten Themen extrahiert. Dies soll, basierend auf dem TF-IDF-Vektor auf zwei Arten durchgeführt werden:

- Non-Negative Matrix Factorization (NMF)
- Latent Dirichlet Allocation (LDA)

Zuerst wird jede Komponente initialisiert und anschließend jeweils auf dem TF-IDF-Vektor angewandt. Die jeweiligen gefundenen Themen werden im Anschluss daran ausgegeben.


In [ ]:
nmf = NMF(n_components=2, init='random', random_state=0)
nmf_topic = nmf.fit_transform(tfidf)

print("\nThemen, die von NMF gefunden wurden:")
for topic_idx, topic in enumerate(nmf.components_):
    print("Thema %d:" % (topic_idx))
    print(" ".join([vectorizer.get_feature_names_out()[i]
                    for i in topic.argsort()[:-10 - 1:-1]]))

lda = LatentDirichletAllocation(n_components=2, max_iter=5, learning_method='online', learning_offset=50., random_state=0)
lda_topic = lda.fit_transform(tfidf)

print("\nThemen, die von LDA gefunden wurden:")
for topic_idx, topic in enumerate(lda.components_):
    print("Thema %d:" % (topic_idx))
    print(" ".join([vectorizer.get_feature_names_out()[i]
                    for i in topic.argsort()[:-10 - 1:-1]]))
